In [12]:
import plotly.io as pio
REPORTS = (BASE / "reports")
REPORTS.mkdir(exist_ok=True)
out = REPORTS / "retail_dashboard.html"
pio.write_html(dashboard, file=str(out), include_plotlyjs="cdn", full_html=True)
out.as_posix()


'/Users/meghanathota/retail_project/reports/retail_dashboard.html'

In [13]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# project paths
BASE = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
GOLD = BASE / "data" / "gold"
REPORTS = BASE / "reports"
REPORTS.mkdir(exist_ok=True)

monthly_dir = GOLD / "monthly_revenue"
cat_dir     = GOLD / "monthly_revenue_by_category"   # optional


In [17]:
def read_spark_csv_folder(folder: Path) -> pd.DataFrame:
    files = list(folder.glob("*.csv")) or list(folder.rglob("*.csv"))
    if not files:
        raise FileNotFoundError(f"No CSV files in {folder}")
    return pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

def to_month_index(df, year_col="year", month_col="month"):
    df[year_col]  = df[year_col].astype(int)
    df[month_col] = df[month_col].astype(int)
    df["date"]    = pd.to_datetime(df[year_col].astype(str) + "-" + df[month_col].astype(str) + "-01")
    return df.sort_values("date")

def format_money(x):
    if pd.isna(x): return "-"
    if abs(x) >= 1_000_000:
        return f"${x/1_000_000:.2f}M"
000:
        return f"${x/1_000:.1f}K"
    return f"${x:,.0f}"



In [18]:
monthly = read_spark_csv_folder(monthly_dir)
monthly.columns = [c.strip().lower() for c in monthly.columns]
monthly = to_month_index(monthly, "year", "month")
monthly = monthly.rename(columns={"total_sales":"revenue"})
monthly = monthly[["date","year","month","revenue"]]

# optional by-category
try:
    monthly_cat = read_spark_csv_folder(cat_dir)
    monthly_cat.columns = [c.strip().lower() for c in monthly_cat.columns]
    # figure out category column name
    cat_col = next((c for c in monthly_cat.columns if c not in {"year","month","total_sales"}), None)
    if cat_col:
        monthly_cat = to_month_index(monthly_cat, "year", "month")
        monthly_cat = monthly_cat.rename(columns={"total_sales":"revenue"})
        monthly_cat = monthly_cat[["date","year","month",cat_col,"revenue"]]
    else:
        monthly_cat = None
except FileNotFoundError:
    monthly_cat = None

monthly.head()


,date,year,month,revenue
0,2011-01-01,2011,1,277864.0
1,2011-02-01,2011,2,168114.0
2,2011-03-01,2011,3,261895.0
3,2011-04-01,2011,4,182323.0
4,2011-05-01,2011,5,182691.0


In [22]:
# fill missing months (so charts are smooth)
midx = pd.period_range(monthly["date"].min(), monthly["date"].max(), freq="M").to_timestamp()
monthly = monthly.set_index("date").reindex(midx).rename_axis("date").reset_index()
monthly["year"]  = monthly["date"].dt.year
monthly["month"] = monthly["date"].dt.month
monthly["revenue"] = monthly["revenue"].fillna(0.0)

# KPIs
last = monthly.iloc[-1]["revenue"]
prev = monthly.iloc[-2]["revenue"] if len(monthly) > 1 else np.nan
last_year = monthly.iloc[-13]["revenue"] if len(monthly) > 12 else np.nan

mom = (last - prev) / prev * 100 if prev and prev != 0 else np.nan
yoy = (last - last_year) / last_year * 100 if last_year and last_year != 0 else np.nan
ytd = monthly[monthly["date"].dt.year == monthly["date"].max().year]["revenue"].sum()

# anomalies and rolling mean
window = 6
monthly["ma"]   = monthly["revenue"].rolling(window, min_periods=3).mean()
monthly["std"]  = monthly["revenue"].rolling(window, min_periods=3).std()
monthly["z"]    = (monthly["revenue"] - monthly["ma"]) / monthly["std"]
monthly["anomaly"] = (monthly["z"].abs() >= 2.0)

monthly["band_low"]  = (monthly["ma"] - monthly["std"]).clip(lower=0)
monthly["band_high"] = (monthly["ma"] + monthly["std"])



In [23]:
# sparkline
sparkline = go.Figure(go.Scatter(
    x=monthly["date"], y=monthly["revenue"], mode="lines",
    hoverinfo="x+y", name="Revenue"
))
sparkline.update_layout(margin=dict(l=0,r=0,t=0,b=0), height=80, showlegend=False)
sparkline.update_yaxes(visible=False); sparkline.update_xaxes(visible=False)

kpi_text = {
    "Revenue (latest)": format_money(last),
    "MoM": (f"{mom:+.1f}%" if pd.notna(mom) else "-"),
    "YoY": (f"{yoy:+.1f}%" if pd.notna(yoy) else "-"),
    "YTD": format_money(ytd),
}
kpi_text


{'Revenue (latest)': '$1.31M',
 'MoM': '+26.6%',
 'YoY': '+71.3%',
 'YTD': '$8.06M'}

In [24]:
fig_trend = go.Figure()

# band
fig_trend.add_traces([
    go.Scatter(
        x=monthly["date"], y=monthly["band_high"], line=dict(width=0), hoverinfo="skip",
        showlegend=False, name="Upper Band"
    ),
    go.Scatter(
        x=monthly["date"], y=monthly["band_low"], line=dict(width=0), fill="tonexty",
        hoverinfo="skip", name="Confidence Band", opacity=0.2
    )
])

# actual
fig_trend.add_trace(go.Scatter(
    x=monthly["date"], y=monthly["revenue"], mode="lines+markers",
    name="Revenue", hovertemplate="%{x|%b %Y}<br>$%{y:,.0f}<extra></extra>"
))

# anomalies
anoms = monthly[monthly["anomaly"] & monthly["ma"].notna()]
if not anoms.empty:
    fig_trend.add_trace(go.Scatter(
        x=anoms["date"], y=anoms["revenue"], mode="markers",
        marker=dict(size=10, color="crimson", symbol="diamond"),
        name="Anomaly",
        hovertemplate="⚠ %{x|%b %Y}<br>$%{y:,.0f}<extra></extra>"
    ))

fig_trend.update_layout(
    title="Monthly Revenue (with Confidence Band & Anomalies)",
    yaxis_title="Revenue", xaxis_title=None, height=420
)


In [25]:
fig_cats = None
fig_area = None

if monthly_cat is not None:
    latest_date = monthly_cat["date"].max()
    cat_col = next(c for c in monthly_cat.columns if c not in {"date","year","month","revenue"})
    top_latest = (
        monthly_cat[monthly_cat["date"] == latest_date]
        .sort_values("revenue", ascending=False).head(10)
    )
    fig_cats = px.bar(
        top_latest, x=cat_col, y="revenue",
        title=f"Top Categories — {latest_date.strftime('%b %Y')}"
    )
    fig_cats.update_yaxes(title="Revenue")

    # stacked area share
    pivot = monthly_cat.pivot_table(index="date", columns=cat_col, values="revenue", aggfunc="sum").fillna(0)
    pivot = pivot.loc[:, pivot.sum().sort_values(ascending=False).index[:8]]  # top 8 only
    fig_area = go.Figure()
    for c in pivot.columns:
        fig_area.add_trace(go.Scatter(x=pivot.index, y=pivot[c], mode="lines",
                                      stackgroup="one", name=c))
    fig_area.update_layout(title="Category Contribution Over Time", height=320)


In [26]:
# dashboard layout
if fig_cats and fig_area:
    dashboard = make_subplots(
        rows=3, cols=1, shared_xaxes=False,
        vertical_spacing=0.08,
        subplot_titles=("Monthly Revenue", "Top Categories (Latest Month)", "Category Contribution Over Time")
    )
    for tr in fig_trend.data: dashboard.add_trace(tr, row=1, col=1)
    for tr in fig_cats.data:  dashboard.add_trace(tr, row=2, col=1)
    for tr in fig_area.data:  dashboard.add_trace(tr, row=3, col=1)
    dashboard.update_layout(height=1000, showlegend=True)
elif fig_cats:
    dashboard = make_subplots(
        rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.1,
        subplot_titles=("Monthly Revenue", "Top Categories (Latest Month)")
    )
    for tr in fig_trend.data: dashboard.add_trace(tr, row=1, col=1)
    for tr in fig_cats.data:  dashboard.add_trace(tr, row=2, col=1)
    dashboard.update_layout(height=820, showlegend=True)
else:
    dashboard = fig_trend

# add KPI annotations at the top
kpi_annotations = [
    dict(x=0.0, y=1.12, xref="paper", yref="paper", text=f"<b>Revenue</b>: {kpi_text['Revenue (latest)']}", showarrow=False, align="left"),
    dict(x=0.35, y=1.12, xref="paper", yref="paper", text=f"<b>MoM</b>: {kpi_text['MoM']}", showarrow=False),
    dict(x=0.55, y=1.12, xref="paper", yref="paper", text=f"<b>YoY</b>: {kpi_text['YoY']}", showarrow=False),
    dict(x=0.75, y=1.12, xref="paper", yref="paper", text=f"<b>YTD</b>: {kpi_text['YTD']}", showarrow=False),
]
dashboard.update_layout(annotations=(dashboard.layout.annotations or []) + tuple(kpi_annotations))

# show in notebook
dashboard.show()

# export
outfile = REPORTS / "retail_dashboard.html"
pio.write_html(dashboard, file=str(outfile), include_plotlyjs="cdn", full_html=True)
print("Saved →", outfile)


Saved → /Users/meghanathota/retail_project/reports/retail_dashboard.html


In [27]:
import plotly.io as pio

# Minimal, modern base template
pio.templates["retail_clean"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family="Inter,Roboto,Arial,Helvetica,sans-serif", size=13),
        margin=dict(l=40, r=20, t=50, b=40),
        paper_bgcolor="white",
        plot_bgcolor="white",
        xaxis=dict(showgrid=True, gridcolor="rgba(0,0,0,0.05)"),
        yaxis=dict(showgrid=True, gridcolor="rgba(0,0,0,0.05)"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
        hoverlabel=dict(bgcolor="white")
    )
)
pio.templates.default = "retail_clean"

def money_tickformat():
    # ticks like $1.2K / $1.2M
    return "$~s"

def add_currency_axis(fig, axis="y"):
    if axis == "y":
        fig.update_yaxes(tickformat=money_tickformat())
    else:
        fig.update_xaxes(tickformat=money_tickformat())
    return fig



In [28]:
# Optional YoY overlay (align last 12 months vs previous 12)
if len(monthly) >= 24:
    base = monthly.tail(24).copy()
    base["series"] = ["Last 12M"]*12 + ["Prev 12M"]*12
    base["pos"] = list(range(12))*2
    yoy_overlay = True
else:
    yoy_overlay = False

fig_trend = go.Figure()

# Smoothed confidence band (clip negatives, lighter fill)
fig_trend.add_traces([
    go.Scatter(
        x=monthly["date"], y=monthly["band_high"], line=dict(width=0),
        hoverinfo="skip", showlegend=False
    ),
    go.Scatter(
        x=monthly["date"], y=monthly["band_low"], line=dict(width=0),
        fill="tonexty", fillcolor="rgba(0, 120, 255, 0.10)",
        hoverinfo="skip", showlegend=False
    )
])

# Revenue line with markers
fig_trend.add_trace(go.Scatter(
    x=monthly["date"], y=monthly["revenue"], mode="lines+markers",
    name="Revenue", line=dict(width=2),
    hovertemplate="<b>%{x|%b %Y}</b><br>$%{y:,.0f}<extra></extra>"
))

# Anomaly markers
anoms = monthly[monthly["anomaly"] & monthly["ma"].notna()]
if not anoms.empty:
    fig_trend.add_trace(go.Scatter(
        x=anoms["date"], y=anoms["revenue"], mode="markers",
        marker=dict(size=10, color="crimson", symbol="diamond"),
        name="Anomaly",
        hovertemplate="⚠ %{x|%b %Y}<br>$%{y:,.0f}<extra></extra>"
    ))

# YoY overlay as offset series (optional)
if yoy_overlay:
    last12 = base[base["series"]=="Last 12M"]
    prev12 = base[base["series"]=="Prev 12M"]
    fig_trend.add_trace(go.Scatter(
        x=last12["pos"], y=last12["revenue"], mode="lines",
        name="Last 12M (Indexed)", line=dict(width=2, dash="solid"),
        hovertemplate="Month +%{x}<br>$%{y:,.0f}<extra></extra>", visible="legendonly"
    ))
    fig_trend.add_trace(go.Scatter(
        x=prev12["pos"], y=prev12["revenue"], mode="lines",
        name="Prev 12M (Indexed)", line=dict(width=2, dash="dot"),
        hovertemplate="Month +%{x}<br>$%{y:,.0f}<extra></extra>", visible="legendonly"
    ))

fig_trend.update_layout(
    title="Monthly Revenue",
    xaxis=dict(rangeslider=dict(visible=True), rangeselector=dict(
        buttons=[
            dict(count=6, step="month", stepmode="backward", label="6M"),
            dict(count=12, step="month", stepmode="backward", label="12M"),
            dict(step="all", label="All")
        ]
    )),
    height=450
)
add_currency_axis(fig_trend, "y")


In [29]:
# Build only if by-category table is available
fig_cats = None
if monthly_cat is not None:
    cat_col = next(c for c in monthly_cat.columns if c not in {"date","year","month","revenue"})
    latest_date = monthly_cat["date"].max()
    top_latest = (monthly_cat[monthly_cat["date"] == latest_date]
                  .groupby(cat_col, as_index=False)["revenue"].sum()
                  .sort_values("revenue", ascending=False))
    # Pareto calc: cumulative share
    top_latest["cum_share"] = top_latest["revenue"].cumsum() / top_latest["revenue"].sum()
    top10 = top_latest.head(15)  # show 15; adjust as you like

    fig_cats = go.Figure()
    fig_cats.add_trace(go.Bar(
        x=top10[cat_col], y=top10["revenue"], name="Revenue",
        hovertemplate=f"%{{x}}<br>$%{{y:,.0f}}<extra></extra>"
    ))
    fig_cats.add_trace(go.Scatter(
        x=top10[cat_col], y=top10["cum_share"]*100, name="Pareto %",
        mode="lines+markers", yaxis="y2",
        hovertemplate="Cumulative: %{y:.1f}%<extra></extra>"
    ))
    fig_cats.update_layout(
        title=f"Top Categories — {latest_date:%b %Y}",
        height=420,
        yaxis=dict(title="Revenue"),
        yaxis2=dict(title="Cumulative %", overlaying="y", side="right", rangemode="tozero", tickformat=".0f")
    )
    add_currency_axis(fig_cats, "y")


In [30]:
fig_area = None
if monthly_cat is not None:
    cat_col = next(c for c in monthly_cat.columns if c not in {"date","year","month","revenue"})
    pivot = (monthly_cat.pivot_table(index="date", columns=cat_col, values="revenue", aggfunc="sum")
             .fillna(0))
    top_cols = pivot.sum().sort_values(ascending=False).index[:8]
    pivot = pivot[top_cols]

    fig_area = go.Figure()
    for c in pivot.columns:
        fig_area.add_trace(go.Scatter(
            x=pivot.index, y=pivot[c], mode="lines", stackgroup="one", name=c,
            hovertemplate="<b>%{x|%b %Y}</b><br>$%{y:,.0f}<extra></extra>"
        ))
    fig_area.update_layout(title="Category Contribution Over Time", height=360)
    add_currency_axis(fig_area, "y")


In [31]:
fig_map = None
try:
    # try from Silver if you saved country there; else skip
    # If you have a gold-by-country table, read that instead.
    # This quick attempt aggregates from monthly_cat if it contains 'country' (unlikely),
    # otherwise it just won't run.
    if monthly_cat is not None and "country" in monthly_cat.columns:
        by_cty = (monthly_cat.groupby("country", as_index=False)["revenue"].sum())
        fig_map = px.choropleth(
            by_cty, locations="country", locationmode="country names",
            color="revenue", title="Revenue by Country (Total)",
            color_continuous_scale="Blues"
        )
        fig_map.update_layout(height=420)
        add_currency_axis(fig_map, "color")  # colorbar format
except Exception:
    fig_map = None


In [32]:
# Simple insights from last 2 periods and latest category movers
insights = []

# MoM / YoY statements
if pd.notna(mom):
    insights.append(f"Revenue {'increased' if mom>=0 else 'decreased'} {abs(mom):.1f}% vs last month.")
if pd.notna(yoy):
    insights.append(f"Revenue {'increased' if yoy>=0 else 'decreased'} {abs(yoy):.1f}% vs last year.")

# Top contributors this month (if categories available)
if monthly_cat is not None:
    cat_col = next(c for c in monthly_cat.columns if c not in {"date","year","month","revenue"})
    latest = monthly_cat[monthly_cat["date"] == monthly_cat["date"].max()]
    top3 = (latest.groupby(cat_col, as_index=False)["revenue"].sum()
                 .sort_values("revenue", ascending=False).head(3))
    if not top3.empty:
        plist = ", ".join([f"{r[cat_col]} ({format_money(r['revenue'])})" for _, r in top3.iterrows()])
        insights.append(f"Top contributors this month: {plist}.")

# Turn into HTML annotation
insight_html = "<br>".join([f"• {s}" for s in insights]) if insights else "• No major changes detected."


In [ ]:
from plotly.subplots import make_subplots

# Build layout slots based on what exists
rows = []
if True: rows.append(("Revenue Trend", fig_trend))
if fig_cats is not None: rows.append(("Top Categories + Pareto", fig_cats))
if fig_area is not None: rows.append(("Category Share Over Time", fig_area))
if fig_map  is not None: rows.append(("Geo Revenue (Optional)", fig_map))

dashboard = make_subplots(
    rows=len(rows), cols=1, shared_xaxes=False, vertical_spacing=0.08,
    subplot_titles=[t for t,_ in rows]
)
for i, (_, fig) in enumerate(rows, start=1):
    for tr in fig.data:
        dashboard.add_trace(tr, row=i, col=1)

dashboard.update_layout(height=380*len(rows) + 120, showlegend=True)

# KPI badges as annotations
kpi_annotations = [
    dict(x=0.00, y=1.08, xref="paper", yref="paper", text=f"<b>Revenue</b>: {format_money(last)}", showarrow=False),
    dict(x=0.33, y=1.08, xref="paper", yref="paper", text=f"<b>MoM</b>: {('+' if mom>=0 else '')+f'{mom:.1f}%'}" if pd.notna(mom) else "<b>MoM</b>: -", showarrow=False),
    dict(x=0.56, y=1.08, xref="paper", yref="paper", text=f"<b>YoY</b>: {('+' if yoy>=0 else '')+f'{yoy:.1f}%'}" if pd.notna(yoy) else "<b>YoY</b>: -", showarrow=False),
    dict(x=0.78, y=1.08, xref="paper", yref="paper", text=f"<b>YTD</b>: {format_money(ytd)}", showarrow=False),
    dict(x=0.00, y=1.15, xref="paper", yref="paper",
         text="<b>Retail Performance Dashboard</b>", showarrow=False, font=dict(size=18)),
    dict(x=0.00, y=1.02, xref="paper", yref="paper",
         text=insight_html, showarrow=False, align="left")
]
dashboard.update_layout(annotations=(dashboard.layout.annotations or []) + tuple(kpi_annotations))

# Export
REPORTS.mkdir(exist_ok=True)
outfile = REPORTS / "retail_dashboard_polished.html"
pio.write_html(dashboard, file=str(outfile), include_plotlyjs="cdn", full_html=True)
print("Saved →", outfile)


Saved → /Users/meghanathota/retail_project/reports/retail_dashboard_polished.html
